# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!


  1. Use RAGAS to Generate Synthetic Data
  2. Load them into a LangSmith Dataset
  3. Evaluate our RAG chain against the synthetic test data
  4. Make changes to our pipeline
  5. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Mark
[nltk_data]     Gelson\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mark Gelson\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [5]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [6]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [7]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [13]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.6 MB 1.4 MB/s eta 0:00:01
   ------------------------- -------------- 1.0/1.6 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 2.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 8c51aab4-10bc-4f7e-9821-e6022ac03696 does not have a summary. Skipping filtering.
Node c46f0828-c93b-449a-9009-60952444e96b does not have a summary. Skipping filtering.
Node 202c92aa-4535-4968-8178-680dcea02b90 does not have a summary. Skipping filtering.
Node 990ca7f7-c3e9-450e-8e36-7dc76c00fc4d does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 137)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 137)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

</div>


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :

* SingleHopSpecificQuerySynthesizer – Generates simple, one-step questions answerable from a single, specific piece of data.
* MultiHopAbstractQuerySynthesizer – Creates multi-step questions that connect several pieces of data, but in a general or abstract way.
* MultiHopSpecificQuerySynthesizer – Produces multi-step questions that are concrete and fact-specific, pointing to particular details in the data.
</span>


Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the role of SENATE in AI regulation?,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The SENATE is involved in regulating the devel...,single_hop_specifc_query_synthesizer
1,What are the concerns associated with Artifici...,[AI presents enormous opportunities for the Ph...,The concerns regarding ASI include the possibi...,single_hop_specifc_query_synthesizer
2,What does the Philippine Constitution say abou...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The provided context does not include specific...,single_hop_specifc_query_synthesizer
3,What is AGI?,"[1 \na) Promote innovation, technological adva...",The context states that the Act shall regulate...,single_hop_specifc_query_synthesizer
4,How is Artificial General Intelligence charact...,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Artificial General Intelligence (AGI) refers t...,single_hop_specifc_query_synthesizer
5,How do the regulations and risk classification...,[<1-hop>\n\n1 \nSec. 15. AI Ethics Review Boar...,The regulations established by the AI Ethics R...,multi_hop_abstract_query_synthesizer
6,How do ai job classification standards and imp...,[<1-hop>\n\n1 \nd) Take full responsibility fo...,The ai job classification standards involve de...,multi_hop_abstract_query_synthesizer
7,How does the proposed Philippine AI regulation...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,The proposed Philippine AI regulation framewor...,multi_hop_abstract_query_synthesizer
8,what does the artificial intelligence regulati...,[<1-hop>\n\nTWENTIETH CONGRESS OF THE \nREPUBL...,the artificial intelligence regulation act (AI...,multi_hop_specific_query_synthesizer
9,How does the jurisdiction of the National AI C...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,The NAIC has exclusive jurisdiction over all m...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 361c867d-2169-49ed-977c-376289940677 does not have a summary. Skipping filtering.
Node 019ecbdd-d5a7-4a7e-8e95-fa012e15c1da does not have a summary. Skipping filtering.
Node d8e76b8e-3d77-4cfb-8360-070394afc25f does not have a summary. Skipping filtering.
Node 4a78a842-9c32-456f-b381-c460dd2c040f does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What are the key features and implications of ...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The European Union AI Act of 2024 is the world...,single_hop_specifc_query_synthesizer
1,What are the concerns associated with Artifici...,[AI presents enormous opportunities for the Ph...,The concerns include the possibility that Arti...,single_hop_specifc_query_synthesizer
2,What is the focus of the TWENTIETH CONGRESS re...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,"The TWENTIETH CONGRESS of the Philippines, thr...",single_hop_specifc_query_synthesizer
3,How do the Philipines' policies on AI aim to b...,"[1 \na) Promote innovation, technological adva...",The policies in the Philippines aim to strike ...,single_hop_specifc_query_synthesizer
4,H0w do the cirumstances that incresing penalti...,"[<1-hop>\n\n1 \nendorsements, voice recordings...",The context shows that penalties are increased...,multi_hop_abstract_query_synthesizer
5,How do the legal consequences for allowing har...,[<1-hop>\n\n1 \niii) Mandatory compliance trai...,The context indicates that platforms or publis...,multi_hop_abstract_query_synthesizer
6,How does the review of AI-related employment t...,[<1-hop>\n\n1 \niii) \nProof of employer engag...,The review of AI-related employment trends inv...,multi_hop_abstract_query_synthesizer
7,How AI promote sectors like health and agricul...,[<1-hop>\n\n1 \nSec. 8. NAICSecretariat. - The...,The context shows that the NAIC is responsible...,multi_hop_abstract_query_synthesizer
8,How does the DICT's role in AI regulation rela...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,The DICT is responsible for the regulation and...,multi_hop_specific_query_synthesizer
9,"How does the NAIC's regulation of AI, as outli...",[<1-hop>\n\n1 \nd) Take full responsibility fo...,"The NAIC's regulation of AI, including its res...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "Philippines AI Bills"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to create or disseminate disinformation is a fine of One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000), or imprisonment of three (3) years to ten (10) years, or both, at the discretion of the court.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: 
- `labeled_helpfulness_evaluator`:
</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* qa_evaluator: Evaluates whether the model’s answer is correct compared to the expected answer, focusing on factual accuracy.
* labeled_helpfulness_evaluator: Evaluates how helpful the model’s response is to the user, considering the correct reference answer as context.
</div>

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'fixed-food-31' at:
https://smith.langchain.com/o/165d8a20-5e3b-4980-90ce-33bbd9644a7a/datasets/f4f2ee27-3d20-4eca-819f-cc8026f96349/compare?selectedSessions=ae1e27ba-d4af-4ed4-ace7-dcdbb0c8fc03




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the DOST relate to the NAIC's jurisdi...,Based on the provided context:\n\nThe National...,None,The context indicates that the NAIC has exclus...,1,1,0,7.462796,40287736-bfb6-490f-9a42-671f3a1940d9,014268db-0d8d-437a-8d9e-df4539427928
1,How does the NAIC's jurisdiction over AI devel...,Based on the provided context:\n\nSec. 6 state...,None,The NAIC has exclusive jurisdiction over all m...,1,1,0,5.949579,192c58e1-3f7f-41b3-a32c-df144f5710ae,0e642ff6-a0a5-447c-a3f4-6ff00a3d1a7a
2,"How does the NAIC's regulation of AI, as outli...","Based on the provided context, the NAIC (Natio...",None,"The NAIC's regulation of AI, including its res...",1,0,0,11.486868,eb95abd2-4db2-467c-aa58-19ed44be87bd,5df95029-5846-4516-bbff-4ba113857c0b
3,How does the DICT's role in AI regulation rela...,I don't know.,None,The DICT is responsible for the regulation and...,0,0,0,2.336158,ac0224f2-75dd-4d79-94da-cc4c2eaf1573,677804ec-24d8-40c1-9349-6ce0ad0b06b3
4,How AI promote sectors like health and agricul...,Based on the provided context:\n\nAI promotes ...,None,The context shows that the NAIC is responsible...,1,1,0,6.934977,0f6a641d-a2b0-4242-8da7-5893852c39e6,9e9b3993-1837-4d6a-a6ab-5456118d4715
5,How does the review of AI-related employment t...,I don't know.,None,The review of AI-related employment trends inv...,0,0,0,2.106791,778eb7d3-e9c0-40c4-a9c4-17d19a950f2a,7e03c62e-0b59-44a7-95e4-20cf15915314
6,How do the legal consequences for allowing har...,Based on the provided context:\n\n1. **Neglige...,None,The context indicates that platforms or publis...,1,1,0,11.670685,ea4f531f-088e-4366-b759-d33da7d59ba2,d0f16ad1-0b60-47df-987b-581851184c66
7,H0w do the cirumstances that incresing penalti...,I don't know.,None,The context shows that penalties are increased...,0,0,0,2.273663,a4977a98-9c8c-4ca9-b85b-f8d98fc6030b,7f6b3b62-62be-4826-9bc2-291bf36e2341
8,How do the Philipines' policies on AI aim to b...,The Philippines' policies on AI aim to balance...,None,The policies in the Philippines aim to strike ...,1,1,0,5.211534,3043ce0c-6896-47aa-9142-f3fa849b51b6,c274589c-ee5b-468c-bb1e-9d0cf76ac206
9,What is the focus of the TWENTIETH CONGRESS re...,"Based on the context provided, the focus of th...",None,"The TWENTIETH CONGRESS of the Philippines, thr...",1,1,0,3.621943,1556637f-4829-4d1f-a675-1000adf28f3e,1febb004-7987-436d-a060-5645cff55195


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### ❓Question #3:

Why would modifying our chunk size modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:

* Changing the chunk size affects how much context each piece of text contains. If chunks are too small, the model may miss important context, leading to incomplete or less accurate answers. If chunks are too large, they might exceed the model’s input limits or include unnecessary information, which can confuse the model and slow down processing. The right chunk size balances context retention with efficiency, improving retrieval accuracy and overall application performance.

</div>

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### ❓Question #4:

Why would modifying our embedding model modify the performance of our application?

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer :
* Changing the embedding model impacts how well text is converted into numerical vectors, which affects how accurately the system can find and match relevant information. A more advanced model can capture deeper meanings and relationships between words, improving search accuracy but possibly increasing cost and processing time. A simpler model may be faster and cheaper but could miss subtle connections, lowering retrieval quality. Choosing the right model balances accuracy, speed, and cost for the application’s needs.

</div>

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG 2"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
empathy_rag_chain.invoke({"question" : "Why is the Philippines AI Bill important?"})

"Thank you for your thoughtful question. Based on the context provided, the Philippines AI Bill is important because it aims to carefully balance encouraging technological innovation with ensuring that AI systems remain safe, ethical, transparent, and under meaningful human oversight. This bill is designed to protect the rights and welfare of every citizen while supporting Filipino ingenuity and addressing national development challenges. It also recognizes the potential risks of AI, such as threats posed by misuse or unintended consequences, and emphasizes the State's responsibility to prevent AI from being used to perpetrate crimes or cause harm. In essence, this bill lays down a general framework to guide the responsible and lawful development of AI in the Philippines, helping to create a secure and inclusive digital future.\n\nI hope this helps clarify why the bill holds such significance, and I appreciate your interest in such an important topic. If you have any more questions or 

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'worthwhile-peace-30' at:
https://smith.langchain.com/o/165d8a20-5e3b-4980-90ce-33bbd9644a7a/datasets/f4f2ee27-3d20-4eca-819f-cc8026f96349/compare?selectedSessions=83a62b9f-6e7d-406b-b688-3ac292a0630f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the DOST relate to the NAIC's jurisdi...,Thank you for your thoughtful question. Based ...,None,The context indicates that the NAIC has exclus...,1,1,1,4.851409,40287736-bfb6-490f-9a42-671f3a1940d9,b3609eb2-d9cf-4bda-8f1c-c3335b0214d3
1,How does the NAIC's jurisdiction over AI devel...,Thank you for your thoughtful question. Based ...,None,The NAIC has exclusive jurisdiction over all m...,1,1,1,5.987636,192c58e1-3f7f-41b3-a32c-df144f5710ae,83fd2098-eee2-4776-9416-3359f0aee789
2,"How does the NAIC's regulation of AI, as outli...",Thank you for your thoughtful question. From t...,None,"The NAIC's regulation of AI, including its res...",1,0,1,6.919561,eb95abd2-4db2-467c-aa58-19ed44be87bd,2f10c6f8-6a05-49b3-8fcd-0a0dc3653e57
3,How does the DICT's role in AI regulation rela...,Thank you for your thoughtful question. Based ...,None,The DICT is responsible for the regulation and...,0,0,1,4.749940,ac0224f2-75dd-4d79-94da-cc4c2eaf1573,442fd064-4430-4adf-8962-c7d69e8671df
4,How AI promote sectors like health and agricul...,Thank you for your thoughtful question. Based ...,None,The context shows that the NAIC is responsible...,1,1,1,6.416858,0f6a641d-a2b0-4242-8da7-5893852c39e6,967ff341-aeac-4038-846b-7faeff6ff1dd
5,How does the review of AI-related employment t...,Thank you for your thoughtful question. From t...,None,The review of AI-related employment trends inv...,1,1,1,5.134889,778eb7d3-e9c0-40c4-a9c4-17d19a950f2a,95b7434a-898e-4cb0-83c7-ee5602a9fad7
6,How do the legal consequences for allowing har...,Thank you for such a thoughtful and important ...,None,The context indicates that platforms or publis...,1,1,1,10.631142,ea4f531f-088e-4366-b759-d33da7d59ba2,5f60f73a-f6a0-4752-9ce3-f8e148addf57
7,H0w do the cirumstances that incresing penalti...,Thank you for your thoughtful question. I unde...,None,The context shows that penalties are increased...,1,1,1,5.895920,a4977a98-9c8c-4ca9-b85b-f8d98fc6030b,fa1e336b-7ceb-4535-bfa7-99b40ec6d6fd
8,How do the Philipines' policies on AI aim to b...,Thank you for your thoughtful question. Based ...,None,The policies in the Philippines aim to strike ...,1,1,1,5.150296,3043ce0c-6896-47aa-9142-f3fa849b51b6,238b08c8-4402-49e9-8f8b-7f592e58f2cf
9,What is the focus of the TWENTIETH CONGRESS re...,Thank you for your thoughtful question. Based ...,None,"The TWENTIETH CONGRESS of the Philippines, thr...",1,1,1,4.058491,1556637f-4829-4d1f-a675-1000adf28f3e,bb1c3036-6836-4bb6-893a-de9b48e2cc64


<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

#### 🏗️ Question #4:

Explain why you believe certain metrics changed in certain ways, and provide a screenshot of the difference between the two chains.

</div>

<div style="background-color: #204B8E; color: white; padding: 10px; border-radius: 5px;">

### Answer:
* The changes in metrics can be attributed to improvements in the updated chain’s retrieval and prompting approach. QA accuracy increased because the revised chain was likely able to fetch more relevant context and frame its responses in a way that better matched the ground truth answers. Helpfulness also improved, possibly due to refinements in prompt instructions that encouraged more complete and actionable responses. The rise in empathy scores suggests that the new chain incorporated more user-focused and understanding language, likely from explicit tone guidance in the prompt. Minor decreases in some areas, such as conciseness, may have resulted from prioritizing completeness and empathy over brevity. The screenshot below shows the side-by-side comparison of the two chains, highlighting these metric differences.

</div>

![Description](ComparisonScreenshot.png)